In [3]:
import os
from z3 import *

def parse_dat_file(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
        
        m = int(lines[0].strip())
        n = int(lines[1].strip())
        l_values = list(map(int, lines[2].strip().split()))
        s_values = list(map(int, lines[3].strip().split()))
        distance_matrix = [list(map(int, line.strip().split())) for line in lines[4:]]
        
        return {
            'm': m,
            'n': n,
            'l_values': l_values,
            's_values': s_values,
            'distance_matrix': distance_matrix
        }

def read_dat_files(folder_path):
    dat_files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    instances = {}
    
    for dat_file in dat_files:
        file_path = os.path.join(folder_path, dat_file)
        instances[dat_file] = parse_dat_file(file_path)
    
    return instances

folder_path = 'Instances'  # Adjust this path as necessary
instances = read_dat_files(folder_path)

# Display parsed data for verification
for filename, data in instances.items():
    print(f"Filename: {filename}")
    print(f"m: {data['m']}")
    print(f"n: {data['n']}")
    print(f"l_values: {data['l_values']}")
    print(f"s_values: {data['s_values']}")
    print(f"distance_matrix: {data['distance_matrix']}")
    print()


Filename: inst01.dat
m: 2
n: 6
l_values: [15, 10]
s_values: [3, 2, 6, 5, 4, 4]
distance_matrix: [[0, 3, 4, 5, 6, 6, 2], [3, 0, 1, 4, 5, 7, 3], [4, 1, 0, 5, 6, 6, 4], [4, 4, 5, 0, 3, 3, 2], [6, 7, 8, 3, 0, 2, 4], [6, 7, 8, 3, 2, 0, 4], [2, 3, 4, 3, 4, 4, 0]]

Filename: inst02.dat
m: 6
n: 9
l_values: [190, 185, 185, 190, 195, 185]
s_values: [11, 11, 23, 16, 2, 1, 24, 14, 20]
distance_matrix: [[0, 199, 119, 28, 179, 77, 145, 61, 123, 87], [199, 0, 81, 206, 38, 122, 55, 138, 76, 113], [119, 81, 0, 126, 69, 121, 26, 117, 91, 32], [28, 206, 126, 0, 186, 84, 152, 68, 130, 94], [169, 38, 79, 176, 0, 92, 58, 108, 46, 98], [77, 122, 121, 84, 102, 0, 100, 16, 46, 96], [145, 55, 26, 152, 58, 100, 0, 91, 70, 58], [61, 138, 113, 68, 118, 16, 91, 0, 62, 87], [123, 76, 91, 130, 56, 46, 70, 62, 0, 66], [87, 113, 32, 94, 94, 96, 58, 87, 66, 0]]

Filename: inst03.dat
m: 3
n: 7
l_values: [15, 10, 7]
s_values: [3, 2, 6, 8, 5, 4, 4]
distance_matrix: [[0, 3, 3, 6, 5, 6, 6, 2], [3, 0, 6, 3, 4, 7, 7, 3], [3, 4

In [ ]:
def solve_mcp(instance, timeout=300000):
    m = instance['m']
    n = instance['n']
    l_values = instance['l_values']
    s_values = instance['s_values']
    D = instance['distance_matrix']
    
    # Define the Z3 variables
    X = [[Bool(f'x_{i}_{j}') for j in range(n)] for i in range(m)]
    courier_distance = [Int(f'courier_distance_{i}') for i in range(m)]
    max_distance = Int('max_distance')
    
    s = Solver()
    s.set("timeout", timeout)  # Set the timeout in milliseconds (300000 ms = 300 seconds)
    
    # Constraints: Each item must be assigned to exactly one courier
    for j in range(n):
        s.add(Sum([If(X[i][j], 1, 0) for i in range(m)]) == 1)
    
    # Constraints: Each courier's load must not exceed its capacity
    for i in range(m):
        s.add(Sum([If(X[i][j], s_values[j], 0) for j in range(n)]) <= l_values[i])
    
    # Constraints: Define the travel times and ensure tours start and end at the origin
    for i in range(m):
        travel_distance = Sum([If(X[i][j], D[n][j] + D[j][n], 0) for j in range(n)])
        s.add(courier_distance[i] == travel_distance)
    
    # Constraint: Max distance is the maximum of all courier distances
    for i in range(m):
        s.add(max_distance >= courier_distance[i])
    
    # Binary search to minimize the maximum distance
    low, high = 0, sum(sum(row) for row in D)  # initial bounds for binary search
    optimal_solution = None

    while low <= high:
        mid = (low + high) // 2
        s.push()
        s.add(max_distance <= mid)
        if s.check() == sat:
            optimal_solution = (mid, s.model())
            high = mid - 1
        else:
            low = mid + 1
        s.pop()
    
    if optimal_solution:
        max_dist, model = optimal_solution
        solution = [[model.evaluate(X[i][j]) for j in range(n)] for i in range(m)]
        return max_dist, solution
    else:
        return None
    

# Solve each instance and print the results
for filename, instance in instances.items():
    result = solve_mcp(instance)
    if result:
        max_distance, solution = result
        print(f"Instance: {filename}")
        print(f"Max Distance: {max_distance}")
        for i, items in enumerate(solution):
            assigned_items = [j+1 for j, assigned in enumerate(items) if is_true(assigned)]
            print(f"Courier {i+1}: Items {assigned_items}")
    else:
        print(f"Instance: {filename} could not be solved within the time limit")

Instance: inst01.dat
Max Distance: 21
Courier 1: Items [3, 4, 6]
Courier 2: Items [1, 2, 5]
Instance: inst02.dat
Max Distance: 306
Courier 1: Items [5]
Courier 2: Items [3, 6]
Courier 3: Items [1, 9]
Courier 4: Items [8]
Courier 5: Items [2]
Courier 6: Items [4, 7]
Instance: inst03.dat
Max Distance: 20
Courier 1: Items [3, 5, 6]
Courier 2: Items [2, 4]
Courier 3: Items [1, 7]
Instance: inst04.dat
Max Distance: 220
Courier 1: Items []
Courier 2: Items [1, 9]
Courier 3: Items [7, 10]
Courier 4: Items [4]
Courier 5: Items [3, 5]
Courier 6: Items [6]
Courier 7: Items [8]
Courier 8: Items [2]
Instance: inst05.dat
Max Distance: 280
Courier 1: Items [2]
Courier 2: Items [1, 3]
Instance: inst06.dat
Max Distance: 322
Courier 1: Items [5]
Courier 2: Items [4]
Courier 3: Items [8]
Courier 4: Items [1, 7]
Courier 5: Items [3]
Courier 6: Items [2, 6]
